This notebook willbe used to explore the tidal ellipses conversions and manipulation at the VENUS nodes.

##Loading and Preparing Data
First we import all the necessary packages

In [23]:
import os
import matplotlib.pylab as plt
import matplotlib.ticker as ticker
from matplotlib.patches import Ellipse
import numpy as np
from IPython.display import display, Math, Latex

import netCDF4 as nc
from scipy.optimize import curve_fit
from salishsea_tools import tidetools
from salishsea_tools import viz_tools
from salishsea_tools.nowcast import research_VENUS

%matplotlib inline


Now we import the bathymetry of the model

In [7]:
grid_B = nc.Dataset('/data/dlatorne/MEOPAR/NEMO-forcing/grid/bathy_meter_SalishSea2.nc')
lats = grid.variables['nav_lat'][:]
lons = grid.variables['nav_lon'][:]
bathy = grid.variables['Bathymetry']

Next, we need the quarter-hourly updated u and v velocities from the model at the VENUS nodes

In [35]:
filec = '/data/dlatorne/MEOPAR/SalishSea/nowcast/20may15/VENUS_central_gridded.nc'
filee = '/data/dlatorne/MEOPAR/SalishSea/nowcast/20may15/VENUS_east_gridded.nc'

grid_c = nc.Dataset(filec)
grid_e = nc.Dataset(filee)

u_u_c = grid_c.variables['vozocrtx']
v_v_c = grid_c.variables['vomecrty']
w_w_c = grid_c.variables['vovecrtz']
dep_t_c = grid_c.variables['depthv']
dep_w_c = grid_c.variables['depthw']
time_c = grid_c.variables['time_counter'][:]/3600.

u_u_e = grid_e.variables['vozocrtx']
v_v_e = grid_e.variables['vomecrty']
w_w_e = grid_e.variables['vovecrtz']
dep_t_e = grid_e.variables['depthv']
dep_w_e = grid_e.variables['depthw']
time_e = grid_e.variables['time_counter'][:]/3600.

We must unstagger and rotate the u and v velocities to have them be evaluated at the same center grid point then rotated to have u as the East/West velocities and v as the North/South velocities.

In [59]:
u_c, v_c=research_VENUS.unstag_rot_gridded(u_u_c, v_v_c, 'Central')
u_e, v_e=research_VENUS.unstag_rot_gridded(u_u_e, v_v_e, 'East')


##Set-up Tide Information
We will allocate space for the tidal components M2 and K1 for each u and c and set the frequencies of these two tidal component, these are already known.

In [62]:
M2freq = 28.984106 # degrees per hour
M2freq = M2freq*np.pi/180. # radians per hour
K1freq = 15.041069*np.pi/180.

thesize=40

vM2amp = np.zeros(thesize); vM2pha = np.zeros(thesize)
vK1amp = np.zeros(thesize); vK1pha = np.zeros(thesize)
uM2amp = np.zeros(thesize); uM2pha = np.zeros(thesize)
uK1amp = np.zeros(thesize); uK1pha = np.zeros(thesize)

Now this is the function we will try to fit to our model:

In [49]:
display(Math(r'u = mean + A_{M2}cos(w_{M2}t-\theta_{M2}) + A_{K1}cos(w_{K1}t-\theta_{K1})'))
display(Math(r'v = mean + A_{M2}cos(w_{M2}t-\theta_{M2}) + A_{K1}cos(w_{K1}t-\theta_{K1})'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

This assumes the current is only affected by M2 and K1 which is not true but a good approsimation because they are the two major components at the chosen locations

In [50]:
# function for fit, this assumes two tides only
def double(x, M2amp, M2pha, K1amp, K1pha, mean):
    return (mean + M2amp*np.cos(M2freq*x-M2pha*np.pi/180.)+
            K1amp*np.cos(K1freq*x-K1pha*np.pi/180.))

In [63]:
# function to do the fitting and actually find the tidal components
def fittit_ (uaus, vaus, time, dj=1):
    for dep in np.arange(1,40):
        if vaus[:,:].any() != 0.:
            fitted, cov = curve_fit(double,time[:],vaus[:,dep]) 
            if fitted[0] < 0:
                fitted[0] = -fitted[0]
                fitted[1] = fitted[1]+180.
            if fitted[1] > 180:
                fitted[1] = fitted[1] - 360.
            elif fitted[1] < -180-360:
                fitted[1] = fitted[1] + 720.
            elif fitted [1] < -180:
                fitted[1] = fitted[1] + 360.
            if fitted[2] < 0:
                fitted[2] = -fitted[2]
                fitted[3] = fitted[3]+180.
            vM2amp[dep] = fitted[0]
            vM2pha[dep] = fitted[1]
            vK1amp[dep] = fitted[2]
            vK1pha[dep] = fitted[3]
    for dep in np.arange(1,40):
        if uaus[:,:].any() !=0.:
            fitted, cov = curve_fit(double,time[:],uaus[:,dep]) 
            if fitted[0] < 0:
                fitted[0] = -fitted[0]
                fitted[1] = fitted[1]+180.
            if fitted[1] > 180+360:
                fitted[1] = fitted[1] -720
            elif fitted[1] > 180:
                fitted[1] = fitted[1] - 360.
            elif fitted[1] < -180-360:
                fitted[1] = fitted[1] + 720.
            elif fitted [1] < -180:
                fitted[1] = fitted[1] + 360.
            if fitted[2] < 0:
                fitted[2] = -fitted[2]
                fitted[3] = fitted[3]+180.
            uM2amp[dep] = fitted[0]
            uM2pha[dep] = fitted[1]
            uK1amp[dep] = fitted[2]
            uK1pha[dep] = fitted[3]
        return vM2amp, vM2pha, vK1amp, vK1pha, uM2amp, uM2pha, uK1amp, uK1pha

Now we will run the fit for the Central and East nodes

In [68]:
vM2amp, vM2pha, vK1amp, vK1pha, uM2amp, uM2pha, uK1amp, uK1pha = fittit (u_c, v_c, 
                                                                         time_c)

print uM2amp[22],uM2pha[22],vM2amp[22],vM2pha[22]

0.0 0.0 0.161329963644 -36.9148475965
